## References

In [8]:
# https://radimrehurek.com/gensim/tut1.html
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# https://docs.python.org/2/library/re.html

## Prepare Notebook

In [9]:
# import packages
from gensim import corpora
import pandas as pd
import logging
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
import io
import pickle

In [10]:
# download stopwords and lemmatizer from nltk package
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to C:\Users\Sebastian
[nltk_data]     Birk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sebastian
[nltk_data]     Birk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# log events
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Load and Inspect Dataset

In [12]:
# read data with timestamp as index
tweets = pd.read_csv("tweets.csv", encoding="latin1", parse_dates=True, 
                     index_col="created", usecols=range(1,28))

In [13]:
# inspect dataframe
tweets.head()

,text,favoriteCount,replyToSN,truncated,replyToSID,replyToUID,statusSource,retweetCount,longitude,latitude,...,retweetCountOutlier,tweetcount,movement,language3,dayofweek,weeknumber,month,idBarrio_xy,idBarrio,user
created,,,,,,,,,,,,,,,,,,,,,
2017-11-28 22:44:07,I'm at El Raval in Barcelona https://t.co/bSGA...,0,NaN,False,NaN,NaN,"<a href=""http://foursquare.com"" rel=""nofollow""...",0,2.168964,41.380936,...,0,1,1.0,OTHER,Tuesday,48,November,1,55,u03883
2017-11-22 19:48:53,<ed><U+00A0><U+00BC><ed><U+00B6><U+0098> @ O't...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.168180,41.381031,...,0,2,1.0,OTHER,Wednesday,47,November,1,55,u02046
2017-11-21 21:58:48,Aquesta setmana publiquem una nova escapada al...,1,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",1,2.168721,41.380217,...,0,1,1.0,CATALAN,Tuesday,47,November,1,55,u03884
2017-11-20 11:15:10,I'm at El Raval in Barcelona https://t.co/xz2A...,0,NaN,False,NaN,NaN,"<a href=""http://foursquare.com"" rel=""nofollow""...",0,2.168964,41.380936,...,0,4,1.0,OTHER,Monday,47,November,1,55,u00881
2017-11-20 10:08:51,Hablan catalán y es importante destacar que el...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.168180,41.381031,...,0,2,1.0,SPANISH,Monday,47,November,1,55,u02047


In [14]:
# display dataframe info
tweets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23778 entries, 2017-11-28 22:44:07 to 2017-06-11 15:55:42
Data columns (total 26 columns):
text                    23778 non-null object
favoriteCount           23778 non-null int64
replyToSN               821 non-null object
truncated               23778 non-null bool
replyToSID              574 non-null float64
replyToUID              821 non-null float64
statusSource            23778 non-null object
retweetCount            23778 non-null int64
longitude               23778 non-null float64
latitude                23778 non-null float64
id_seccion              23778 non-null int64
horaPeticion            23778 non-null object
id_distrito             23778 non-null int64
grupoHora               23778 non-null object
id_seccion_xy           23778 non-null int64
favoriteCountOutlier    23778 non-null int64
retweetCountOutlier     23778 non-null int64
tweetcount              23778 non-null int64
movement                23778 non-null f

In [15]:
# describe dataframe 
tweets.describe()

,favoriteCount,replyToSID,replyToUID,retweetCount,longitude,latitude,id_seccion,id_distrito,id_seccion_xy,favoriteCountOutlier,retweetCountOutlier,tweetcount,movement,weeknumber,idBarrio_xy,idBarrio
count,23778.000000,5.740000e+02,8.210000e+02,23778.000000,23778.000000,23778.000000,2.377800e+04,23778.000000,2.377800e+04,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000
mean,1.200774,9.015351e+17,4.078693e+16,0.285222,2.171064,41.395325,8.019040e+08,801903.985953,7.999476e+08,0.000673,0.000126,17.686349,0.690218,34.522962,18.136008,21.180503
std,20.403217,5.293076e+16,1.808261e+17,4.610914,0.021190,0.014936,3.028198e+03,3.023254,3.955736e+07,0.025932,0.011232,62.044925,0.326073,7.046783,21.415965,22.107374
min,0.000000,1.229677e+17,7.802900e+05,0.000000,2.059243,41.332580,8.019010e+08,801901.000000,0.000000e+00,0.000000,0.000000,1.000000,0.100000,23.000000,0.000000,1.000000
25%,0.000000,8.862349e+17,1.195479e+08,0.000000,2.159720,41.382780,8.019020e+08,801902.000000,8.019020e+08,0.000000,0.000000,1.000000,0.384615,28.000000,6.000000,6.000000
50%,0.000000,9.079774e+17,3.537928e+08,0.000000,2.174778,41.395250,8.019021e+08,801902.000000,8.019021e+08,0.000000,0.000000,3.000000,0.750000,34.000000,7.000000,9.000000
75%,1.000000,9.232741e+17,1.028215e+09,0.000000,2.176944,41.404080,8.019060e+08,801906.000000,8.019050e+08,0.000000,0.000000,8.000000,1.000000,40.000000,25.000000,31.000000
max,2449.000000,9.354775e+17,9.290872e+17,567.000000,2.226620,41.465590,8.019102e+08,801910.000000,8.019102e+08,1.000000,1.000000,440.000000,1.000000,49.000000,73.000000,73.000000


In [16]:
# divide dataset according to language: extract english language
english_tweets = tweets[tweets["language3"] == "ENGLISH"].copy()

## Data Cleaning and Preparation

### Preprocess Twitter Text

In [17]:
# display text
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

english_tweets["text"]

created
2017-11-18 19:12:28    Brew Pub to try a few of the 30 beers on offer <ed><U+00A0><U+00BD><ed><U+00B8><U+0080> #olgodbarcelona #ølgod #triathlontraining
 https://t.co/X3TK9OnJjs                                                                                                                                                                                                                                         
2017-11-27 10:46:20    "Art is coming face to face with yourself" <ed><U+00A0><U+00BD><ed><U+00B9><U+0083><ed><U+00A0><U+00BD><ed><U+00B9><U+0082> @ Parc de les Tres Xemeneies https://t.co/FedFTbDc6E                                                                                                                                                                                                                   
2017-11-25 20:44:52    Today's quickie @ Parc de les Tres Xemeneies https://t.co/ASKSgYfks8                                                                               

In [18]:
# remove links
english_tweets["text_clean"] = english_tweets["text"].str.replace(r"http\S+", "")
# remove emoticons
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"<.*>", "")
# remove punctuation, special characters etc.
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"&amp", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\.", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\,", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\;", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\-", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\"", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\\\\", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\/", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\*", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"@", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\n", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\|", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"W//", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"!", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"~", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r")", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"(", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"?", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r":", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\{", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\}", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"_", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"h:[0-9]+m:[0-9]+s", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[0-9]+", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"w/", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x97]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x96]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x95]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x94]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x93]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x92]+", "'")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x91]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x85]+", "")

# reduce white spaces to 1
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r" +", " ")

In [19]:
# display cleaned text
english_tweets["text_clean"]

created
2017-11-18 19:12:28    Brew Pub to try a few of the beers on offer #olgodbarcelona #ølgod #triathlontraining                             
2017-11-27 10:46:20    Art is coming face to face with yourself Parc de les Tres Xemeneies                                               
2017-11-25 20:44:52    Today's quickie Parc de les Tres Xemeneies                                                                        
2017-11-23 10:47:18    All hail the freshest seafood in town Let's feast #hadthemostmusselsinmylife #tripoftheyear                       
2017-11-19 22:30:26    Dance to the beat of your own drum #urban #music #live #stage #graffiti #streetart                                
2017-11-19 10:37:25    I just finished walking km in h m s with #Endomondo #endorphins                                                   
2017-11-26 11:24:48    #blackfriday shopping #friends Then #bday party for Mallan great food margaritas and more                         
2017-11-25 18:52:42    Ano

In [20]:
# reorder columns
english_tweets.columns
cols = ['text', 'text_clean', 'favoriteCount', 'replyToSN', 'truncated', 'replyToSID',
       'replyToUID', 'statusSource', 'retweetCount', 'longitude', 'latitude',
       'id_seccion', 'horaPeticion', 'id_distrito', 'grupoHora',
       'id_seccion_xy', 'favoriteCountOutlier', 'retweetCountOutlier',
       'tweetcount', 'movement', 'language3', 'dayofweek', 'weeknumber',
       'month', 'idBarrio_xy', 'idBarrio', 'user']

english_tweets = english_tweets[cols]

#### In order to train the topic models, 3 different pooling methods for the creation of documents are used: No Pooling (1), User Pooling (2) and Hashtag Pooling (3).

### Training Documents Option 1 (No Pooling)

In [21]:
# treat every tweet as a different document (no pooling)
documents = english_tweets["text_clean"].tolist()

### Training Documents Option 2 (User Pooling)

In [22]:
# treat all tweets by one user as one single document (user pooling)
user_combined = english_tweets[["text_clean","user"]].groupby("user")["text_clean"].apply(lambda x: "".join(x))
documents_user_pooling = user_combined.tolist()

### Training Documents Option 3 (Hashtag Pooling)

In [23]:
# treat all tweets with the same hashtag as one single document (hashtag pooling)

# find all hashtags
english_tweets["hashtags"] = english_tweets["text_clean"].str.findall(r'#.*?(?=\s|$)')

# separate hashtags in columns
hashtags_tweets = pd.DataFrame(english_tweets["hashtags"].tolist(),
                               columns=["hashtag1", "hashtag2", "hashtag3", "hashtag4",
                                        "hashtag5", "hashtag6", "hashtag7", "hashtag8",
                                        "hashtag9", "hashtag10", "hashtag11", "hashtag12",
                                        "hashtag13"])

# join hashtags with tweet text
hashtags_tweets.index = english_tweets.index
hashtags_tweets = english_tweets.join(hashtags_tweets)

In [24]:
# create one dataframe with text for each hashtag column and save them in a dictionary
dict = {}
for index, item in enumerate(["hash1", "hash2", "hash3", "hash4", "hash5",
                              "hash6", "hash7", "hash8", "hash9", "hash10",
                              "hash11", "hash12", "hash13"]):
    dict[item] = hashtags_tweets[["hashtag" + str(index + 1), "text_clean"]].copy()
    dict[item].columns = ["hashtag", "text"]
    dict[item].dropna(inplace=True)

# concatenate all dataframes to one dataframe (the result is a dataframe
# where there is text for each hashtag found)
hashtags = pd.DataFrame()
for item in dict:
    hashtags = pd.concat([hashtags, dict[item]])

# combine text for each hashtag
hashtags_combined = hashtags.groupby("hashtag")["text"].apply(lambda x: "".join(x))

In [25]:
# remove some generic hashtags that cover a lot of different topics
hashtags_combined.drop(["#Barcelona", "#Catalunya", "#Spain", "#BCN", "#BARCELONA",
                        "#Espana", "#BarcelonaSpain"], inplace=True)

In [26]:
# create documents
documents_hashtag_pooling = hashtags_combined.tolist()

### Analyze Documents

In [ ]:
# calculate number of terms per document
total = []
for item in documents:
    total.append(len(item.split()))

In [ ]:
# calculate total number of terms
sum = 0
for number in total:
    sum += number

In [ ]:
# calculate average number of terms per document
sum/len(documents)

### Prepare Test Documents

#### The trained topic models will then be used to determine the topics of test documents.

#### The first objective of the research is to analyze the distribution of topics over the districts. For this purpose, district pooling is used to create the documents that will be tested.

In [27]:
# merge all tweets from each district (district pooling) and treat them as one single document respectively
district_combined = english_tweets[["text_clean","id_distrito"]].groupby("id_distrito")["text_clean"].apply(lambda x: "".join(x))
documents_district_pooling = district_combined.tolist()

In [21]:
# check documents
district_combined

id_distrito
801901    #cure Betty Ford's in Barcelona Sunday funday wishing a could skate like a pro #iceskating #ice #winter #fun Winter Where en Black and White Christine The Benefit Concert Nov was a wrap Dreams come true if you work hard for it Mine came  Insider Tips on Tickets Bar by foodieling in Travel with Ling #TicketsBar #Barcelona #brodasbros #artistas #top #recomano anar #cracks #hiphop #tribute #jamesbrown #teatrecondal Plaça Espanya in #Barcelona #Spain #Europe #art #travel #instatravel #instatrip #tour What a night dondiablo #DonDiablo #fcbarcelona #opium #barcelona #spain #hexagon Los Martes Now #languageexchange #intercambiolinguistico #funeventsbcn #meetingpeople preview #kettal #luxury #outdoor #lifestyle #barcelona #made #dining #chair #sofa #chaise Croissant crisis in France #watercolor #winsorandnewton Art i Col lec Claudia Vives Fierro  La check list para gestionar #SocialMedia ¿añadiríais alguna tarea #RedesSociales Spanish trails #photography #barcelona #night

In [31]:
# check if order is right
district_combined.index # correct

Int64Index([801901, 801902, 801903, 801904, 801905, 801906, 801907, 801908, 801909, 801910], dtype='int64', name='id_distrito')

#### The second objective is to look at the dynamic topic development over time. For this purpose, the dataset is divided according to time and documents are created on this basis.

#### Divide dataframe according to month

In [32]:
# sort index
sorted_tweets = english_tweets.sort_index()

In [33]:
# check first and last date
print(sorted_tweets.index[0]) # June 2017
print(sorted_tweets.index[-1]) # December 2017 (very incomplete)

2017-06-11 13:46:35
2017-12-04 21:20:25


In [34]:
# create column that contains the month of the tweets
sorted_tweets['month'] = sorted_tweets.index.month

In [37]:
# split dataframe according to month
june = sorted_tweets.loc['2017-06-01':'2017-06-30']
july = sorted_tweets.loc['2017-07-01':'2017-07-31']
august = sorted_tweets.loc['2017-08-01':'2017-08-31']
september = sorted_tweets.loc['2017-09-01':'2017-09-30']
october = sorted_tweets.loc['2017-10-01':'2017-10-31']
november = sorted_tweets.loc['2017-11-01':'2017-11-30']
december = sorted_tweets.loc['2017-12-01':'2017-12-31']

In [38]:
# count number of tweets per month
len(june)

1270

In [45]:
# merge all tweets from each month and treat them as one document respectively
months_combined = english_tweets[["text_clean","month"]].groupby("month")["text_clean"].apply(lambda x: "".join(x))
documents_month_pooling = months_combined.tolist()

In [43]:
# check how many documents
len(documents_month_pooling) # should be 7

7

In [79]:
# check order of documents
months_combined.index # Aug, Dec, Jul, Jun, Nov, Oct, Sep

Index(['August', 'December', 'July', 'June', 'November', 'October', 'September'], dtype='object', name='month')

In [95]:
# count number of characters
len(documents_month_pooling[6])

102928

In [85]:
# convert strings to wordlist
aug_words = documents_month_pooling[0].split()
dec_words = documents_month_pooling[1].split()
jul_words = documents_month_pooling[2].split()
jun_words = documents_month_pooling[3].split()
nov_words = documents_month_pooling[4].split()
oct_words = documents_month_pooling[5].split()
sept_words = documents_month_pooling[6].split()

In [88]:
# count number of words per month
print(len(jun_words))
print(len(jul_words))
print(len(aug_words))
print(len(sept_words))
print(len(oct_words))
print(len(nov_words))
print(len(dec_words))

13426
18927
12578
15347
12436
8855
380


In [101]:
# count unique words
print(len(set(jun_words)))
print(len(set(jul_words)))
print(len(set(aug_words)))
print(len(set(sept_words)))
print(len(set(oct_words)))
print(len(set(nov_words)))
print(len(set(dec_words)))

4691
6007
4416
5083
4707
3783
248


In [74]:
# merge all tweets from one district and one month and treat them as one document respectively
districts_per_month_combined = sorted_tweets[["text_clean","month","id_distrito"]].groupby(["month","id_distrito"])["text_clean"].apply(lambda x: "".join(x))
documents_district_per_month_pooling = districts_per_month_combined.tolist()

In [81]:
# check order of documents
districts_per_month_combined.index 

MultiIndex(levels=[[6, 7, 8, 9, 10, 11, 12], [801901, 801902, 801903, 801904, 801905, 801906, 801907, 801908, 801909, 801910]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 6, 7]],
           names=['month', 'id_distrito'])

In [50]:
# check how many documents
len(documents_district_per_month_pooling)

62

In [30]:
# inspect dataframe
districts_per_month_combined

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Preprocess Documents for NMF Topic Modeling Method

In [51]:
# create copy of no pooling documents
nmf_documents = list(documents)

In [52]:
# transform to lower case
for doc_idx, doc in enumerate(nmf_documents):
    nmf_documents[doc_idx] = nmf_documents[doc_idx].lower()

In [53]:
# delete stopwords
for doc_idx, doc in enumerate(nmf_documents):
    nmf_documents[doc_idx] = doc.replace(" year ", " ").replace(" to ", " ").replace(" on ", " ").replace(" wa ", " ").replace(" #yourup ", " ").replace(" de ", " ").replace(" just ", " ").replace(" posted ", " ").replace(" photo ", " ").replace(" la ", " ").replace(" del ", " ").replace(" en ", " ").replace(" los ", " ").replace(" el ", " ").replace(" las ", " ").replace(" barcelona ", " ").replace(" #bcn ", " ").replace(" just ", " ").replace(" cada ", " ").replace(" nuestra ", " ").replace(" around ", " ").replace(" spanish ", " ").replace(" día ", " ").replace(" dia ", " ").replace(" #photo ", " ").replace(" first ", " ").replace(" thing ", " ").replace(" last ", " ").replace(" #spain ", " ").replace(" carrer ", " ").replace(" make ", " ").replace(" &lt ", " ").replace(" &gt ", " ").replace(" de ", " ").replace(" for ", " ").replace(" a ", " ").replace(" of ", " ").replace(" the ", " ").replace(" and ", " ").replace(" to ", " ").replace(" in ", " ").replace(" at ", " ").replace(" by ", " ").replace(" one ", " ").replace(" day ", " ").replace(" get ", " ").replace(" españa ", " ").replace(" #españa ", " ").replace(" #repost ", " ").replace(" since ", " ").replace(" still ", " ").replace(" never ", " ").replace(" thank ", " ").replace(" two ", " ").replace(" think ", " ").replace(" could ", " ").replace(" many ", " ").replace(" even ", " ").replace(" the ", " ").replace(" igers ", " ").replace(" que ", " ").replace(" many ", " ").replace(" con ", " ").replace(" un ", " ").replace(" wa ", " ").replace(" bcn ", " ").replace(" d'horta ", " ").replace(" ever ", " ").replace(" come ", " ").replace(" #ig ", " ").replace(" el ", " ").replace(" i'm ", " ").replace(" i've ", " ").replace(" always ", " ").replace(" le ", " ").replace(" what's ", " ").replace(" #barcelone ", " ").replace(" like ", " ").replace(" last ", " ").replace(" back ", " ").replace(" thanks ", " ").replace(" #barna ", " ").replace(" spain ", " ").replace(" yo ", " ").replace(" #yo ", " ").replace(" el ", " ").replace(" #el ", " ").replace(" barcelona ", " ").replace(" #barcelona ", " ")    

In [54]:
# display preprocessed documents
nmf_documents

['brew pub try few beers offer #olgodbarcelona #ølgod #triathlontraining ',
 'art is coming face face with yourself parc les tres xemeneies ',
 "today's quickie parc les tres xemeneies ",
 "all hail freshest seafood town let's feast #hadthemostmusselsinmylife #tripoftheyear ",
 'dance beat your own drum #urban #music #live #stage #graffiti #streetart ',
 'i finished walking km h m s with #endomondo #endorphins ',
 '#blackfriday shopping #friends then #bday party mallan great food margaritas more ',
 'another why not pastisseria ',
 'yesterday i got new hairstyle with best hair artist city i cut mychell ',
 'good night tienes don jesusisnard #ootd margot house ',
 'stay warm pic x jesusisnard #sunday margot house ',
 'there is nothing better than waking up this saturdays margothousebarcelona wearing ',
 'drinking pumpkin catalanbrewery cocovailbh ',
 'drinking leyenda cervezadougalls cocovailbh ',
 'barcelonas night clinicasden #event #night #night #clinicasden #sonrisas ',
 'brama cros

### Save Training and Test Documents

In [55]:
with io.open('documents.txt', 'w', encoding='utf-8') as f:
    for item in documents:
        f.write(item + "\n")
with io.open('documents_user_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_user_pooling:
        f.write(item + "\n")
with io.open('documents_hashtag_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_hashtag_pooling:
        f.write(item + "\n")
with io.open('documents_district_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_pooling:
        f.write(item + "\n")
with io.open('documents_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_month_pooling:
        f.write(item + "\n")
with io.open('documents_district_per_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_per_month_pooling:
        f.write(item + "\n")
        
with open('documents_no_pooling.p', 'wb') as fp:
    pickle.dump(documents, fp)
    
with open('documents_user_pooling.p', 'wb') as fp:
    pickle.dump(documents_user_pooling, fp)
    
with open('documents_hashtag_pooling.p', 'wb') as fp:
    pickle.dump(documents_hashtag_pooling, fp)
    
with open('nmf_documents_no_pooling.p', 'wb') as fp:
    pickle.dump(nmf_documents, fp)

In [56]:
nmf_documents

['brew pub try few beers offer #olgodbarcelona #ølgod #triathlontraining ',
 'art is coming face face with yourself parc les tres xemeneies ',
 "today's quickie parc les tres xemeneies ",
 "all hail freshest seafood town let's feast #hadthemostmusselsinmylife #tripoftheyear ",
 'dance beat your own drum #urban #music #live #stage #graffiti #streetart ',
 'i finished walking km h m s with #endomondo #endorphins ',
 '#blackfriday shopping #friends then #bday party mallan great food margaritas more ',
 'another why not pastisseria ',
 'yesterday i got new hairstyle with best hair artist city i cut mychell ',
 'good night tienes don jesusisnard #ootd margot house ',
 'stay warm pic x jesusisnard #sunday margot house ',
 'there is nothing better than waking up this saturdays margothousebarcelona wearing ',
 'drinking pumpkin catalanbrewery cocovailbh ',
 'drinking leyenda cervezadougalls cocovailbh ',
 'barcelonas night clinicasden #event #night #night #clinicasden #sonrisas ',
 'brama cros

### Tokenize Training Documents

In [57]:
# we can simply tokenize by space thanks to the previous preprocessing
texts_no_pooling = [[word for word in document.lower().split()]
          for document in documents]

texts_user_pooling = [[word for word in document.lower().split()]
          for document in documents_user_pooling]

texts_hashtag_pooling = [[word for word in document.lower().split()]
          for document in documents_hashtag_pooling]

### Save Unpreprocessed Tokenized Training Documents

In [58]:
with open('tokenized_documents_no_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_no_pooling, fp)
    
with open('tokenized_documents_user_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_user_pooling, fp)
    
with open('tokenized_documents_hashtag_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_hashtag_pooling, fp)

### Further Preprocessing of Training Documents after Tokenization

In [59]:
# remove numbers, but not words that contain numbers.
texts_no_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_hashtag_pooling]

In [60]:
# remove words that are only one character.
texts_no_pooling = [[token for token in doc if len(token) > 1] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if len(token) > 1] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if len(token) > 1] for doc in texts_hashtag_pooling]

In [61]:
# lemmatize all words in all documents.
lemmatizer = WordNetLemmatizer()
texts_no_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_no_pooling]
texts_user_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_user_pooling]
texts_hashtag_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_hashtag_pooling]

In [62]:
# # ignore this part! computing bigrams did not improve models but made them worse!

# # compute bigrams
# # add bigrams and trigrams to docs (only ones that appear 5 times or more)
# bigram = Phrases(texts_no_pooling, min_count=10)
# for idx in range(len(texts_no_pooling)):
#     for token in bigram[texts_no_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_no_pooling[idx].append(token)
#             
# bigram = Phrases(texts_user_pooling, min_count=10)
# for idx in range(len(texts_user_pooling)):
#     for token in bigram[texts_user_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_user_pooling[idx].append(token)
#             
# bigram = Phrases(texts_hashtag_pooling, min_count=10)
# for idx in range(len(texts_hashtag_pooling)):
#     for token in bigram[texts_hashtag_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_hashtag_pooling[idx].append(token)

## Save Tokenized Training Documents

In [63]:
with open('tokenized_documents_no_pooling.p', 'wb') as fp:
    pickle.dump(texts_no_pooling, fp)
    
with open('tokenized_documents_user_pooling.p', 'wb') as fp:
    pickle.dump(texts_user_pooling, fp)
    
with open('tokenized_documents_hashtag_pooling.p', 'wb') as fp:
    pickle.dump(texts_hashtag_pooling, fp)

## Refine and Vectorize Corpora

In [64]:
# define function to refine and vectorize corpus 
# (remove stopwords, very frequent and very infrequent words etc.)

# define stopwords
stpwords = "for a of the and to in at by one #yo #el day get españa #yourup #españa #repost yo el since still never thank two think could many even ha igers th que con un wa bcn d'horta ever come #ig el i'm i've always le what's #barcelone like last back thanks #barna spain barcelona #barcelona cada nuestra around spanish día dia #photo first thing last #spain carrer make &lt &gt de la del en las barcelona #bcn just posted photo year wa".split()

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, 
                additional_stopwords=set(stpwords),
                no_below=2, no_above=0.5,
                dictionary_name='tourism.dict', corpus_name='tourism.mm'):
    print('Building dictionary...')
    dictionary = corpora.Dictionary(docs)
    stopwords = nltk_stopwords().union(additional_stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.compactify()
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    dictionary.compactify()
    dictionary.save(dictionary_name)  # store the dictionary, for future reference
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    corpora.MmCorpus.serialize(corpus_name, corpus)  # store to disk, for later use
    
    return (corpus, dictionary)

In [65]:
# run function to vectorize corpora
corpus_no_pooling = prep_corpus(texts_no_pooling,
                                dictionary_name="tourism_no_pooling.dict",
                                corpus_name="tourism_no_pooling.mm")[0]
dictionary_no_pooling = prep_corpus(texts_no_pooling,
                                    dictionary_name="tourism_no_pooling.dict",
                                    corpus_name="tourism_no_pooling.mm")[1]

corpus_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[0]
dictionary_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[1]

corpus_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[0]
dictionary_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[1]

2018-10-05 07:05:26,049 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Building dictionary...


2018-10-05 07:05:26,264 : INFO : built Dictionary(15443 unique tokens: ['#olgodbarcelona', '#triathlontraining', '#ølgod', 'beer', 'brew']...) from 7633 documents (total 78801 corpus positions)
2018-10-05 07:05:26,323 : INFO : discarding 10451 tokens: [('#olgodbarcelona', 1), ('#triathlontraining', 1), ('quickie', 1), ('#hadthemostmusselsinmylife', 1), ('#tripoftheyear', 1), ('freshest', 1), ('hail', 1), ('mallan', 1), ('mychell', 1), ('tienes', 1)]...
2018-10-05 07:05:26,325 : INFO : keeping 4790 tokens which were in no less than 2 and no more than 3816 (=50.0%) documents
2018-10-05 07:05:26,342 : INFO : resulting dictionary: Dictionary(4790 unique tokens: ['#ølgod', 'beer', 'brew', 'offer', 'pub']...)
2018-10-05 07:05:26,353 : INFO : saving Dictionary object under tourism_no_pooling.dict, separately None
2018-10-05 07:05:26,360 : INFO : saved tourism_no_pooling.dict
2018-10-05 07:05:26,471 : INFO : storing corpus in Matrix Market format to tourism_no_pooling.mm
2018-10-05 07:05:26,47

Building corpus...


2018-10-05 07:05:26,637 : INFO : PROGRESS: saving document #6000
2018-10-05 07:05:26,678 : INFO : PROGRESS: saving document #7000
2018-10-05 07:05:26,696 : INFO : saved 7633x4790 matrix, density=0.101% (37016/36562070)
2018-10-05 07:05:26,698 : INFO : saving MmCorpus index to tourism_no_pooling.mm.index
2018-10-05 07:05:26,707 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


...

2018-10-05 07:05:29,568 : INFO : PROGRESS: saving document #2000
2018-10-05 07:05:29,678 : INFO : PROGRESS: saving document #3000
2018-10-05 07:05:29,781 : INFO : PROGRESS: saving document #4000
2018-10-05 07:05:29,836 : INFO : PROGRESS: saving document #5000
2018-10-05 07:05:29,882 : INFO : PROGRESS: saving document #6000
2018-10-05 07:05:29,886 : INFO : saved 6040x9330 matrix, density=0.140% (78970/56353200)
2018-10-05 07:05:29,889 : INFO : saving MmCorpus index to tourism_hashtag_pooling.mm.index
2018-10-05 07:05:29,896 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Building dictionary...


2018-10-05 07:05:30,144 : INFO : built Dictionary(11318 unique tokens: ['#aj', '#amoramargo', '#aperitivo', '#architecture', '#art']...) from 6040 documents (total 146413 corpus positions)
2018-10-05 07:05:30,176 : INFO : discarding 1797 tokens: [("advisor's", 1), ('armada', 1), ('asistidos', 1), ('barrica', 1), ('bcnshowroom', 1), ('cangilon', 1), ('carot', 1), ('comenzamos', 1), ('cryyou', 1), ('cumpliendo', 1)]...
2018-10-05 07:05:30,179 : INFO : keeping 9330 tokens which were in no less than 2 and no more than 3020 (=50.0%) documents
2018-10-05 07:05:30,200 : INFO : resulting dictionary: Dictionary(9330 unique tokens: ['#aj', '#amoramargo', '#aperitivo', '#architecture', '#art']...)
2018-10-05 07:05:30,209 : INFO : saving Dictionary object under tourism_hashtag_pooling.dict, separately None
2018-10-05 07:05:30,219 : INFO : saved tourism_hashtag_pooling.dict
2018-10-05 07:05:30,394 : INFO : storing corpus in Matrix Market format to tourism_hashtag_pooling.mm
2018-10-05 07:05:30,397 

Building corpus...


2018-10-05 07:05:30,479 : INFO : PROGRESS: saving document #2000
2018-10-05 07:05:30,565 : INFO : PROGRESS: saving document #3000
2018-10-05 07:05:30,667 : INFO : PROGRESS: saving document #4000
2018-10-05 07:05:30,763 : INFO : PROGRESS: saving document #5000
2018-10-05 07:05:30,822 : INFO : PROGRESS: saving document #6000
2018-10-05 07:05:30,825 : INFO : saved 6040x9330 matrix, density=0.140% (78970/56353200)
2018-10-05 07:05:30,828 : INFO : saving MmCorpus index to tourism_hashtag_pooling.mm.index


## Apply Function to Preprocess Test Documents (Before Testing Them with Topic Models)

#### This function has to include all the same steps that were applied to the training documents!

In [66]:
# define function
def preprocess(docs):
    ''' Conduct all preprocessing steps that are conducted to train the LDA model'''
    
    # tokenize documents
    tokenized = [[word for word in document.lower().split()]
          for document in docs]
    
    # remove words that are only one character
    tokenized = [[token for token in doc if len(token) > 1] for doc in tokenized]
    
    # lemmatize all words
    lemmatizer = WordNetLemmatizer()
    lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in tokenized]
    
    # define stopwords
    stpwords = "for a of the and to in at by one #yo #el day get #yourup españa #españa #repost yo el since still never thank two think could many even ha igers th que con un wa bcn d'horta ever come #ig el i'm i've always le what's #barcelone like last back thanks #barna spain barcelona #barcelona cada nuestra around spanish día dia #photo first thing last #spain carrer make &lt &gt de la del en las barcelona #bcn just posted photo year wa".split()
    
    # get stopwords from nltk
    def nltk_stopwords():
        return set(nltk.corpus.stopwords.words('english'))

    # combine stopwords
    stopwords = nltk_stopwords().union(stpwords)
    
    # remove stopwords
    preprocessed = [[token for token in document if token not in stopwords] for document in lemmatized]
    
    return preprocessed

In [67]:
# apply function to test documents
texts_district_pooling = preprocess(documents_district_pooling)
texts_month_pooling = preprocess(documents_month_pooling)
texts_district_per_month_pooling = preprocess(documents_district_per_month_pooling)

## Save Preprocessed Test Documents

In [68]:
with open('tokenized_documents_district_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_pooling, fp)
    
with open('tokenized_documents_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_month_pooling, fp)
    
with open('tokenized_documents_district_per_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_per_month_pooling, fp)

In [69]:
# # ignore this part! just example code!

# # map tokens to ids
# print(dictionary_no_pooling.token2id)
# print(dictionary_user_pooling.token2id)
# print(dictionary_hashtag_pooling.token2id)

In [70]:
# # ignore this part! just example code!

# # convert new document to vector 
# new_doc = "Sagrada Familia is amazing"
# new_vec_no_pooling = dictionary_no_pooling.doc2bow(new_doc.lower().split())
# print(new_vec_no_pooling)

In [71]:
# # ignore this part! not needed for dataset!

# # corpus streaming: one document at a time
# class MyCorpus(object):
#     def __iter__(self):
#         for line in open("corpus_no_pooling.txt"):
#             # assume there's one document per line, tokens separated by whitespace
#             yield dictionary.doc2bow(line.lower().split())
#             
# corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
# print(corpus_memory_friendly)
# 
# for vector in corpus_memory_friendly:  # load one vector into memory at a time
#     print(vector)